Try to find a way of calculating the cost for the YOLO detector without using for loops, this is very costly.

In [208]:
import numpy as np
# from bnr_ml.utils.helpers import meshgrid2D
import theano
from theano import tensor as T
from time import time

In [188]:
def meshgrid2D(arr1, arr2):
    arr1 = arr1.dimshuffle('x',0)
    arr2 = arr2.dimshuffle(0,'x')
    arr1, arr2 = T.repeat(arr1, arr2.shape[0], axis=0), T.repeat(arr2, arr1.shape[1], axis=1)
    return arr1, arr2

In [194]:
S = (7,7)
B = 3
C = 10

In [195]:
N = 10
y_pred= np.random.rand(N,5 * B + C,S[0], S[1]).astype(theano.config.floatX)
# y_truth = np.random.rand(N,4 + C).astype(theano.config.floatX)
y_truth = np.concatenate(
    (
        .25*np.random.rand(N,1),
        .25*np.random.rand(N,1),
        .75*np.random.rand(N,1),
        .75*np.random.rand(N,1),
        np.random.rand(N,C),
    ),
    axis=1
).astype(theano.config.floatX)
# ypred= np.random.rand(10,5 * B + C,S[0], S[1])
# ytruth = np.random.rand(10,4 + C)

In [121]:
output = T.as_tensor(y_pred.astype(theano.config.floatX))
ytruth = T.as_tensor(y_truth.astype(theano.config.floatX))
lmbda_coord = T.as_tensor_variable(5.)
lmbda_noobj = T.as_tensor_variable(.5)

In [122]:
block_height, block_width = 1. / S[0], 1./ S[1]

In [ ]:
offsetx, offsety = np.meshgrid(np.arange(0,1,block_height), np.arange(0,1,block_width))

In [123]:
# theano version
offsetx, offsety = meshgrid2D(T.arange(0,1,block_height), T.arange(0,1,block_width))

In [ ]:
x_idx, y_idx = [i*5 for i in range(B)], [i*5 + 1 for i in range(C)]
w_idx, h_idx = [i*5 + 2 for i in range(B)], [i*5 + 3 for i in range(C)]
conf = [i*5 + 4 for i in range(B)]

In [124]:
# theano version
x_idx, y_idx = T.arange(0,5*B,5), T.arange(1,5*B+ 1, 5)
w_idx, h_idx = T.arange(2,5*B+2,5), T.arange(3,5*B+3,5)
conf_idx = T.arange(4,5*B+4,5)

In [ ]:
ypred[:,x_idx,:,:] += offsetx
ypred[:,y_idx,:,:] += offsety

In [125]:
# theano version
ypred = T.set_subtensor(output[:,x_idx,:,:],output[:,x_idx,:,:] + offsetx)
ypred = T.set_subtensor(output[:,y_idx,:,:],output[:,y_idx,:,:] + offsety)

In [40]:
# calculate the IOU score for each region and box
xi = np.maximum(ypred[:,x_idx,:,:], ytruth[:,0].reshape((10,1,1,1)))
xf = np.minimum(ypred[:,x_idx,:,:] + ypred[:,w_idx,:,:], (ytruth[:,0] + ytruth[:,2]).reshape((10,1,1,1)))

yi = np.maximum(ypred[:,y_idx,:,:], ytruth[:,0].reshape((10,1,1,1)))
yf = np.minimum(ypred[:,y_idx,:,:] + ypred[:,h_idx,:,:], (ytruth[:,0] + ytruth[:,2]).reshape((10,1,1,1)))

In [126]:
# theano version
xi = T.maximum(ypred[:,x_idx,:,:], ytruth[:,0].dimshuffle(0,'x','x','x'))
xf = T.minimum(ypred[:,x_idx,:,:] + ypred[:,w_idx,:,:], (ytruth[:,0] + ytruth[:,2]).dimshuffle(0,'x','x','x'))

yi = T.maximum(ypred[:,y_idx,:,:], ytruth[:,1].dimshuffle(0,'x','x','x'))
yf = T.minimum(ypred[:,y_idx,:,:] + ypred[:,h_idx,:,:], (ytruth[:,1] + ytruth[:,3]).dimshuffle(0,'x','x','x'))

In [ ]:
isec = (xf - xi) * (yf - yi)
union = (ypred[:,w_idx] * ypred[:,h_idx]) + (ytruth[:,2] * ytruth[:,3]).reshape((10,1,1,1)) - isec
iou = np.maximum(isec/union, 0.)

In [127]:
# theano version
isec = (xf - xi) * (yf - yi)
union = (ypred[:,w_idx] * ypred[:,h_idx]) + (ytruth[:,2] * ytruth[:,3]).dimshuffle(0,'x','x','x') - isec
iou = T.maximum(isec/union, 0.)

In [ ]:
cidx, _ = np.meshgrid(range(B),range(10))
# cidx = cidx.reshape((10,2,1,1))
# cidx = np.repeat(np.repeat(cidx,S[0],2),S[1],3)

In [128]:
# theano version
maxval_idx, a = meshgrid2D(T.arange(B), T.arange(ypred.shape[0]))
maxval_idx = maxval_idx.reshape((ypred.shape[0],-1))
maxval_idx = maxval_idx.dimshuffle(0,1,'x','x')
maxval_idx = T.repeat(T.repeat(maxval_idx,S[0],2),S[1],3)

In [ ]:
ismax = np.equal(cidx, iou.argmax(axis=1).reshape((10,1,2,2)))

In [129]:
# theano version
ismax = T.eq(maxval_idx, iou.argmax(axis=1).dimshuffle(0,'x',1,2))

In [ ]:
((iou[ismax] - ypred[:,conf,:,:][ismax])**2).reshape((10,2,2)).shape

In [36]:
width, height = np.ones(S) / S[1], np.ones(S) / S[0]
width, height = width.reshape((1,2,2)), height.reshape((1,2,2))

In [130]:
# theano version
width, height = T.ones(S) / S[1], T.ones(S) / S[0]
width, height = width.dimshuffle('x',0,1), height.dimshuffle('x',0,1)

In [ ]:
xi = np.maximum(width, ytruth[:,0].reshape((10,1,1)))
xf = np.minimum(width + offsetx, (ytruth[:,0] + ytruth[:,2]).reshape((10,1,1)))

yi = np.maximum(height, ytruth[:,1].reshape((10,1,1)))
yf = np.minimum(height + offsety, (ytruth[:,1] + ytruth[:,3]).reshape((10,1,1)))

In [131]:
# theano version
xi = T.maximum(offsetx, ytruth[:,0].dimshuffle(0,'x','x'))
xf = T.minimum(offsetx + width, (ytruth[:,0] + ytruth[:,2]).dimshuffle(0,'x','x'))

yi = T.maximum(offsety, ytruth[:,1].dimshuffle(0,'x','x'))
yf = T.minimum(offsety + height, (ytruth[:,1] + ytruth[:,3]).dimshuffle(0,'x','x'))

In [ ]:
isec = (xf - xi) * (yf - yi)
union = width * height + (ytruth[:,2] * ytruth[:,3]).reshape((10,1,1)) - isec
ioucell = np.maximum(isec / union, 0.)

In [132]:
# theano version
isec = (xf - xi) * (yf - yi)
union = width * height + (ytruth[:,2] * ytruth[:,3]).dimshuffle(0,'x','x') - isec
iou_cell = T.maximum(isec/union, 0.)

In [ ]:
isinter = np.reshape(ioucell > .1, (10,-1,2,2))

In [135]:
# theano version
isinter = (iou_cell > .1).dimshuffle(0,'x',1,2)

In [ ]:
tmp = np.bitwise_and(ismax, isinter)

In [134]:
# theano version
isbox_andcell = T.bitwise_and(ismax, isinter)

In [136]:
# theano
isinter = T.repeat(isinter, C, axis=1)

In [137]:
ypred[:,-C:][isinter.nonzero()].shape.eval()

array([34])

In [140]:
clspred_truth = T.repeat(T.repeat(ytruth[:,-C:].dimshuffle(0,1,'x','x'), S[0], axis=2), S[1], axis=3)

In [143]:
clspred_truth.shape.eval()

array([10,  2,  2,  2])

In [144]:
tmp = T.sum((ypred[:,-C:,:,:][isinter.nonzero()] - clspred_truth[isinter.nonzero()])**2)
print tmp.eval()

6.84959936142


In [151]:




cost = lmbda_coord * T.sum((ypred[:,conf_idx,:,:][isbox_andcell.nonzero()] - iou[isbox_andcell.nonzero()])**2) + \
        lmbda_noobj * T.sum((ypred[:,conf_idx,:,:][T.bitwise_not(isbox_andcell.nonzero())])**2) + \
        T.sum((ypred[:,x_idx,:,:][ismax.nonzero()].reshape((ytruth.shape[0],-1)) - ytruth[:,[0]])**2) + \
        T.sum((ypred[:,y_idx,:,:][ismax.nonzero()].reshape((ytruth.shape[0],-1)) - ytruth[:,[1]])**2) + \
        T.sum((ypred[:,w_idx,:,:][ismax.nonzero()].reshape((ytruth.shape[0],-1)).sqrt() - ytruth[:,[2]].sqrt())**2) + \
        T.sum((ypred[:,h_idx,:,:][ismax.nonzero()].reshape((ytruth.shape[0],-1)).sqrt() - ytruth[:,[3]].sqrt())**2) + \
        T.sum((ypred[:,-C:,:,:][isinter.nonzero()] - clspred_truth[isinter.nonzero()])**2)
        
        
        
        
        
        
        

<p style="font-size:14pt;">This code works so far!</p>

In [204]:
output = T.tensor4('output')
truth = T.matrix('truth')
lmbda_coord = T.scalar('lambda_coord')
lmbda_noobj = T.scalar('lambda_noobj')

In [205]:
cost = get_cost(output, truth, S, B, C)

In [206]:
cost_fn = theano.function([output,truth], cost)

In [207]:
cost_fn(y_pred, y_truth)

array(38624.9765625, dtype=float32)

In [203]:
def get_cost(output, truth, S, B, C, lmbda_coord=5., lmbda_noobj=0.5, iou_thresh=0.1):
    # calculate height/width of individual cell
    block_height, block_width = 1. / S[0], 1./ S[1]

    # get the offset of each cell
    offset_x, offset_y = meshgrid2D(T.arange(0,1,block_height), T.arange(0,1,block_width))
    
    # get indices for x,y,w,h,object-ness for easy access
    x_idx, y_idx = T.arange(0,5*B,5), T.arange(1,5*B+ 1, 5)
    w_idx, h_idx = T.arange(2,5*B+2,5), T.arange(3,5*B+3,5)
    conf_idx = T.arange(4,5*B+4,5)
    
    # Get position predictions with offsets.
    pred = T.set_subtensor(output[:,x_idx,:,:],output[:,x_idx,:,:] + offset_x)
    pred = T.set_subtensor(output[:,y_idx,:,:],output[:,y_idx,:,:] + offset_y)
    
    # Get intersection region bounding box coordinates
    xi = T.maximum(pred[:,x_idx,:,:], truth[:,0].dimshuffle(0,'x','x','x'))
    xf = T.minimum(pred[:,x_idx,:,:] + pred[:,w_idx,:,:], (truth[:,0] + truth[:,2]).dimshuffle(0,'x','x','x'))
    yi = T.maximum(pred[:,y_idx,:,:], truth[:,1].dimshuffle(0,'x','x','x'))
    yf = T.minimum(pred[:,y_idx,:,:] + pred[:,h_idx,:,:], (truth[:,1] + truth[:,3]).dimshuffle(0,'x','x','x'))
    
    # Calculate iou score for predicted boxes and truth
    isec = (xf - xi) * (yf - yi)
    union = (pred[:,w_idx] * pred[:,h_idx]) + (truth[:,2] * truth[:,3]).dimshuffle(0,'x','x','x') - isec
    iou = T.maximum(isec/union, 0.)
    
    # Get index matrix representing max along the 1st dimension for the iou score (reps 'responsible' box).
    maxval_idx, a = meshgrid2D(T.arange(B), T.arange(pred.shape[0]))
    maxval_idx = maxval_idx.reshape((pred.shape[0],-1))
    maxval_idx = maxval_idx.dimshuffle(0,1,'x','x')
    maxval_idx = T.repeat(T.repeat(maxval_idx,S[0],2),S[1],3)
    is_max = T.eq(maxval_idx, iou.argmax(axis=1).dimshuffle(0,'x',1,2))
    
    # Get matrix for the width/height of each cell
    width, height = T.ones(S) / S[1], T.ones(S) / S[0]
    width, height = width.dimshuffle('x',0,1), height.dimshuffle('x',0,1)
    
    # Get bounding box for intersection between CELL and ground truth box.
    xi = T.maximum(offset_x, truth[:,0].dimshuffle(0,'x','x'))
    xf = T.minimum(offset_x + width, (truth[:,0] + truth[:,2]).dimshuffle(0,'x','x'))
    yi = T.maximum(offset_y, truth[:,1].dimshuffle(0,'x','x'))
    yf = T.minimum(offset_y + height, (truth[:,1] + truth[:,3]).dimshuffle(0,'x','x'))
 
    # Calculate iou score for the cell.
    isec = (xf - xi) * (yf - yi)
    union = width * height + (truth[:,2] * truth[:,3]).dimshuffle(0,'x','x') - isec
    iou_cell = T.maximum(isec/union, 0.)
    
    # Get logical matrix representing minimum iou score for cell to be considered overlapping ground truth.
    is_inter = (iou_cell > iou_thresh).dimshuffle(0,'x',1,2)
    
    # Get logical matrix for cells and boxes which overlap and are responsible for prediction.
    isbox_andcell = T.bitwise_and(is_max, is_inter)
    
    # repeat "cell overlaps" logical matrix for the number of classes.
    is_inter = T.repeat(is_inter, C, axis=1)
    
    # repeat the ground truth for class probabilities for each cell.
    clspred_truth = T.repeat(T.repeat(truth[:,-C:].dimshuffle(0,1,'x','x'), S[0], axis=2), S[1], axis=3)
    
    # calculate cost
    cost = lmbda_coord * T.sum((pred[:,conf_idx,:,:][isbox_andcell.nonzero()] - iou[isbox_andcell.nonzero()])**2) + \
        lmbda_noobj * T.sum((pred[:,conf_idx,:,:][T.bitwise_not(isbox_andcell.nonzero())])**2) + \
        T.sum((pred[:,x_idx,:,:][is_max.nonzero()].reshape((truth.shape[0],-1)) - truth[:,[0]])**2) + \
        T.sum((pred[:,y_idx,:,:][is_max.nonzero()].reshape((truth.shape[0],-1)) - truth[:,[1]])**2) + \
        T.sum((pred[:,w_idx,:,:][is_max.nonzero()].reshape((truth.shape[0],-1)).sqrt() - truth[:,[2]].sqrt())**2) + \
        T.sum((pred[:,h_idx,:,:][is_max.nonzero()].reshape((truth.shape[0],-1)).sqrt() - truth[:,[3]].sqrt())**2) + \
        T.sum((pred[:,-C:,:,:][is_inter.nonzero()] - clspred_truth[is_inter.nonzero()])**2)
    
    return cost

In [152]:
cost_fn = theano.function([], cost)

In [153]:
cost_fn()

array(271.6685485839844, dtype=float32)